In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', None)

In [29]:
energy_data = pd.read_csv(r'C:\Users\HP\Desktop\Energy-Consumption-Prediction-RF-ML-main\datasets,\energy_dataset.csv', encoding='UTF-8-SIG')
weather_data = pd.read_csv(r'C:\Users\HP\Desktop\Energy-Consumption-Prediction-RF-ML-main\datasets,\weather_features.csv', encoding='UTF-8-SIG')

In [4]:
print("Energy Dataset:")
energy_data.head()

Energy Dataset:


,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation marine,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,NaN,863.0,1051.0,1899.0,0.0,7096.0,43.0,73.0,49.0,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,NaN,920.0,1009.0,1658.0,0.0,7096.0,43.0,71.0,50.0,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,NaN,1164.0,973.0,1371.0,0.0,7099.0,43.0,73.0,50.0,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,NaN,1503.0,949.0,779.0,0.0,7098.0,43.0,75.0,50.0,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,NaN,1826.0,953.0,720.0,0.0,7097.0,43.0,74.0,42.0,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [5]:
print("Weather Dataset:")
weather_data.head()

Weather Dataset:


,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2015-01-01 00:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
3,2015-01-01 03:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
4,2015-01-01 04:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n


In [6]:
energy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 29 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   time                                         35064 non-null  object 
 1   generation biomass                           35045 non-null  float64
 2   generation fossil brown coal/lignite         35046 non-null  float64
 3   generation fossil coal-derived gas           35046 non-null  float64
 4   generation fossil gas                        35046 non-null  float64
 5   generation fossil hard coal                  35046 non-null  float64
 6   generation fossil oil                        35045 non-null  float64
 7   generation fossil oil shale                  35046 non-null  float64
 8   generation fossil peat                       35046 non-null  float64
 9   generation geothermal                        35046 non-null  float64
 10

In [7]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178396 entries, 0 to 178395
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   dt_iso               178396 non-null  object 
 1   city_name            178396 non-null  object 
 2   temp                 178396 non-null  float64
 3   temp_min             178396 non-null  float64
 4   temp_max             178396 non-null  float64
 5   pressure             178396 non-null  int64  
 6   humidity             178396 non-null  int64  
 7   wind_speed           178396 non-null  int64  
 8   wind_deg             178396 non-null  int64  
 9   rain_1h              178396 non-null  float64
 10  rain_3h              178396 non-null  float64
 11  snow_3h              178396 non-null  float64
 12  clouds_all           178396 non-null  int64  
 13  weather_id           178396 non-null  int64  
 14  weather_main         178396 non-null  object 
 15  weather_descripti

In [8]:
energy_data.columns, weather_data.columns

(Index(['time', 'generation biomass', 'generation fossil brown coal/lignite',
        'generation fossil coal-derived gas', 'generation fossil gas',
        'generation fossil hard coal', 'generation fossil oil',
        'generation fossil oil shale', 'generation fossil peat',
        'generation geothermal', 'generation hydro pumped storage aggregated',
        'generation hydro pumped storage consumption',
        'generation hydro run-of-river and poundage',
        'generation hydro water reservoir', 'generation marine',
        'generation nuclear', 'generation other', 'generation other renewable',
        'generation solar', 'generation waste', 'generation wind offshore',
        'generation wind onshore', 'forecast solar day ahead',
        'forecast wind offshore eday ahead', 'forecast wind onshore day ahead',
        'total load forecast', 'total load actual', 'price day ahead',
        'price actual'],
       dtype='object'),
 Index(['dt_iso', 'city_name', 'temp', 'temp_min',

In [9]:
energy_data.isnull().sum()

time                                               0
generation biomass                                19
generation fossil brown coal/lignite              18
generation fossil coal-derived gas                18
generation fossil gas                             18
generation fossil hard coal                       18
generation fossil oil                             19
generation fossil oil shale                       18
generation fossil peat                            18
generation geothermal                             18
generation hydro pumped storage aggregated     35064
generation hydro pumped storage consumption       19
generation hydro run-of-river and poundage        19
generation hydro water reservoir                  18
generation marine                                 19
generation nuclear                                17
generation other                                  18
generation other renewable                        18
generation solar                              

In [10]:
weather_data.isnull().sum()

dt_iso                 0
city_name              0
temp                   0
temp_min               0
temp_max               0
pressure               0
humidity               0
wind_speed             0
wind_deg               0
rain_1h                0
rain_3h                0
snow_3h                0
clouds_all             0
weather_id             0
weather_main           0
weather_description    0
weather_icon           0
dtype: int64

In [15]:
energy_data['time'] = pd.to_datetime(energy_data['time'])
weather_data['dt_iso'] = pd.to_datetime(weather_data['dt_iso'])

energy_data = energy_data.rename(columns={'time': 'datetime'})
weather_data = weather_data.rename(columns={'dt_iso': 'datetime'})

KeyError: 'time'

In [16]:
merged_data = pd.merge(energy_data, weather_data, on='datetime', how='left')

selected_features = [
    'generation fossil gas',
    'generation nuclear',
    'generation wind onshore',
    'generation solar',
    'temp',
    'pressure',
    'humidity',
    'wind_speed',
    'clouds_all',
    'total_generation'
]

merged_data['total_generation'] = merged_data[[col for col in merged_data.columns if 'generation' in col]].sum(axis=1)
merged_data['energy_demand'] = merged_data['total load actual']
final_data = merged_data[selected_features + ['energy_demand']].copy()

In [17]:
final_data = final_data.fillna(final_data.mean())

print("Missing values after handling:")
print(final_data.isnull().sum())

Missing values after handling:
generation fossil gas      0
generation nuclear         0
generation wind onshore    0
generation solar           0
temp                       0
pressure                   0
humidity                   0
wind_speed                 0
clouds_all                 0
total_generation           0
energy_demand              0
dtype: int64


In [18]:
final_data.head()

,generation fossil gas,generation nuclear,generation wind onshore,generation solar,temp,pressure,humidity,wind_speed,clouds_all,total_generation,energy_demand
0,4844.0,7096.0,6378.0,49.0,270.475000,1001,77,1,0,28251.0,25385.0
1,4844.0,7096.0,6378.0,49.0,267.325000,971,63,1,0,28251.0,25385.0
2,4844.0,7096.0,6378.0,49.0,269.657312,1036,97,0,0,28251.0,25385.0
3,4844.0,7096.0,6378.0,49.0,281.625000,1035,100,7,0,28251.0,25385.0
4,4844.0,7096.0,6378.0,49.0,273.375000,1039,75,1,0,28251.0,25385.0


In [19]:
final_data.shape

(178396, 11)

In [20]:
print(final_data.describe())

       generation fossil gas  generation nuclear  generation wind onshore  \
count          178396.000000       178396.000000            178396.000000   
mean             5626.254428         6263.549316              5471.050963   
std              2203.332157          839.773835              3214.597694   
min                 0.000000            0.000000                 0.000000   
25%              4127.000000         5758.000000              2941.000000   
50%              4973.000000         6561.000000              4857.000000   
75%              6434.000000         7025.000000              7406.000000   
max             20034.000000         7117.000000             17436.000000   

       generation solar           temp      pressure       humidity  \
count     178396.000000  178396.000000  1.783960e+05  178396.000000   
mean        1428.437265     289.618605  1.069261e+03      68.423457   
std         1676.132062       8.026199  5.969632e+03      21.902888   
min            0.00000

In [21]:
final_data.isnull().sum()

generation fossil gas      0
generation nuclear         0
generation wind onshore    0
generation solar           0
temp                       0
pressure                   0
humidity                   0
wind_speed                 0
clouds_all                 0
total_generation           0
energy_demand              0
dtype: int64

In [23]:
X = final_data.drop('energy_demand', axis=1)
y = final_data['energy_demand']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
y_train.head()

109324    25375.0
149113    35031.0
47909     22156.0
160440    35206.0
94216     31753.0
Name: energy_demand, dtype: float64

In [25]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((142716, 10), (142716,), (35680, 10), (35680,))

In [26]:
Lr_Model = LinearRegression()
Lr_Model.fit(X_train, y_train)

LinearRegression()

In [27]:
from sklearn.metrics import mean_squared_error, r2_score

lr_pred = Lr_Model.predict(X_test)
lr_mse = mean_squared_error(y_test, lr_pred)
lr_r2 = r2_score(y_test, lr_pred)

print("Model Performance:")
print("Mean Squared Error:", lr_mse)
print("R2 Score:", lr_r2)

Model Performance:
Mean Squared Error: 5412294.150767416
R2 Score: 0.7422538223289901


In [28]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

In [29]:
rf_pred = rf_model.predict(X_test_scaled)

rf_mse = mean_squared_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

print("Model Performance:")
print("Mean Squared Error:", rf_mse)
print("R2 Score:", rf_r2)

Model Performance:
Mean Squared Error: 218384.77508423623
R2 Score: 0.9896000033494994


In [31]:
import joblib

joblib.dump(rf_model, '../artifacts/energy_demand_model.joblib')
joblib.dump(scaler, '../artifacts/scaler.joblib')

['../artifacts/scaler.joblib']

In [32]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)
print("MSE:", mean_squared_error(y_test, gb_pred))

MSE: 3423159.9154709177


In [28]:
# from xgboost import XGBRegressor

# model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# print("MSE:", mean_squared_error(y_test, y_pred))